In [1]:
#all imports

#import library for regular expression
import re

#import pandas analysis tool
import pandas
import pandas as pd

#import string
import string

#import nltk library
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.tokenize import RegexpTokenizer
from nltk import ngrams

import requests

#import web scraping tool
from bs4 import BeautifulSoup
import os
from IPython.display import HTML

#the pacakge used to remove duplicates in final step of filtering majors.
from fuzzywuzzy.process import dedupe

/Users/yiyangzhou/anaconda/lib/python3.5/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#import path & variables


#import and tokenize resume
resume_path = r"/Users/yiyangzhou/Desktop/Yiyang (Eric) Zhou Resume 2017 Fall.txt"
resume_file = open(resume_path).read()
resume_file2 = open(resume_path).read()
resume_file2 = resume_file2.lower()      #this is a string with the resume file in lower case

#import a list of majors using pandas
major_df = pandas.read_excel('majors.xlsx')
major_df.columns
major_file = major_df['Majors'].values
major_lower = [item.lower() for item in major_file]     #make everything in the major list in lower case

tokenizer = RegexpTokenizer(r'\w+')                #this is a tokenizing tool from nltk
resume_token = tokenizer.tokenize(resume_file)     #tokenize the original resume string 
resume_token2 = tokenizer.tokenize(resume_file2)   #tokenized the lower case resume string


# create those empty of for majors to store results from later major extraction step. 
updated_majors1 = []
indexes_majors1 = []
updated_majors2 = []
indexes_majors2 = []
updated_majors3 = []
indexes_majors3 = []
updated_majors4 = []
indexes_majors4 = []
majors_minors_all = updated_majors1 + updated_majors2 + updated_majors3 + updated_majors4


#import a list of universities
university_df1 = pandas.read_excel('China_University.xlsx')
university_df2 = pandas.read_excel('India_University.xlsx')
university_df3 = pandas.read_excel('US_University.xlsx')
university_file1 = university_df1['Universities'].values
university_file2 = university_df2['Universities'].values
university_file3 = university_df3['Universities'].values
university_lower1 = [item.lower() for item in university_file1]
university_lower2 = [item.lower() for item in university_file2]
university_lower3 = [item.lower() for item in university_file3]
university_combined = university_lower1 + university_lower2 + university_lower3    # this is the final university list that we refer to 


In [19]:
#extract name 

# We use the location of name to do the extraction.


# This function find the first letter on the first line of resume
def extract_first_name(resume):
    name = resume.split('\n', 1)[0]
    first_name = name.split(' ', 1)[0]
    return (first_name)
    print (first_name)

#This function find the last letter on the first line of resume
def extract_last_name(resume):
    name = resume.split('\n', 1)[0]
    last_name = name.split(' ', 1)[-1]
    return (last_name)
    print (last_name)
    
    
#This funciton combines the first name and last name together to get the full name
def extract_name(resume):
    name = extract_first_name(resume_file) + extract_last_name(resume_file)
    print (name)
    
extract_name(resume_file)

Yiyang(Eric)Zhou


In [4]:
#extract email

#extract email based on regular expression. Very straight forward. This works very well, not need to further improvement. 
def extract_email(resume):
    regular_expression = re.compile(r"(\w+[.|\w])*@(\w+[.])*\w+", re.IGNORECASE)
    result = re.search(regular_expression, resume)
    if result:
        result = result.group()
    print (result)

extract_email(resume_file)

yiyang.zhou@rhsmith.umd.edu


In [20]:
#extract phone number

#This function uses regular expression to find any 10 digits consecutive numbers. Any punctuation in between are removed. 
def check_phone_number1(resume):
    resume2 = "".join(c for c in resume if c not in ('!','.','-','(',')',' ','+',))
    result = re.findall(r"\d{10}", resume2)                   
    result = ''.join(result)
    return (result)

#This function uses regular expression to find any 11 digits consecutive numbers. Any punctuation in between are removed.
def check_phone_number2(resume):
    resume2 = "".join(c for c in resume if c not in ('!','.','-','(',')',' ','+',))
    result = re.findall(r"\d{11}", resume2)                   
    result = ''.join(result)
    result = result[1:11]
    return (result)


#This function first call the function that extract 10 digit number. If the result is valid, it returns the result.
#If the first function returns an error, it would call the function to extract the 11 digit number and return the result.
def extract_phone_number(resume):
    try:
        return check_phone_number1(resume) 
        print (check_phone_number1(resume))
    except:
        return check_phone_number2(resume)
        print (check_phone_number2(resume))

extract_phone_number(resume_file)

'6125162045'

In [21]:
#scraping tools

#first get the webpage that we need to scrape
r = requests.get('http://www.4icu.org/us/')

#find the kind of content we need to scrape by looking at the webpage's source code
soup = BeautifulSoup(r.content, "lxml")

#further filter the information we need by looking at the page's source code
letters = soup.find_all(class_="lead")

#create a dictionary of university
university = {}
for element in letters:
    university[element.get_text()] = {}
    
#get the keys for the university dictionary. The keys are university names    
list_keys = [ k for k in university.keys() ]

#organize the university into a data frame
us_university_list = pd.DataFrame(list_keys)
print (us_university_list)

#save to an excel file
'''
writer = pd.ExcelWriter('US_University.xlsx')
df.to_excel(writer, index = False)
writer.save()
'''

                                                      0
0             Southern Illinois University Edwardsville
1                                      Rosemont College
2                                     Taylor University
3                                  Paul Smith's College
4                                    Lourdes University
5                                 College of St. Joseph
6                       North Carolina Wesleyan College
7              California State University, Los Angeles
8           Queens College, City University of New York
9             Maryland University of Integrative Health
10                University of California, Los Angeles
11                                 Augustana University
12                                     Trine University
13                                   University of Iowa
14                            Trinity Christian College
15                                 University of Dayton
16                                    Kalamazoo 

In [7]:
#Extract University
'''
We use the extract matching technique to parse universities. 

First we define series of functions to get n-grams for the resume token and make a list of them.
On later step, we can compare the n-grams resume list to the university list to get the matching result. 
'''

def get_bigrams(input):
    n = 2
    result = []
    bigrams = ngrams(input, n)
    for grams in bigrams:
        x = "%s %s" % grams
        result.append(x)
    return (result)    
    print (result)


test1 = get_bigrams(resume_token)
#print (test1)

def get_threegrams(input):
    n = 3
    result = []
    threegrams = ngrams(input, n)
    for grams in threegrams:
        x = "%s %s %s" % grams
        result.append(x)
    return (result)    
    print (result)

test2 = get_threegrams(resume_token)
#print (test2)

def get_fourgrams(input):
    n = 4
    result = []
    fourgrams = ngrams(input, n)
    for grams in fourgrams:
        x = "%s %s %s %s" % grams
        result.append(x)
    return (result)    
    print (result)

test3 = get_fourgrams(resume_token)
#print (test3)

def get_fivegrams(input):
    n = 5
    result = []
    fivegrams = ngrams(input, n)
    for grams in fivegrams:
        x = "%s %s %s %s %s" % grams
        result.append(x)
    return (result)    
    print (result)

test4 = get_fivegrams(resume_token)
#print (test4)

def get_sixgrams(input):
    n = 6
    result = []
    sixgrams = ngrams(input, n)
    for grams in sixgrams:
        x = "%s %s %s %s %s %s" % grams
        result.append(x)
    return (result)    
    print (result)


def get_university(a,b):
    resume_university=[]
    for x in a:
        if x in b:
            resume_university.append(x)
    return (resume_university)
    print (resume_university)

#combine all the n-grams together    
def extract_university(resume_token_lower,university_combined):
    unigram_university = get_university(resume_token_lower, university_combined)  
    bigram_university = get_university(get_bigrams(resume_token_lower), university_combined)
    threegram_university = get_university(get_threegrams(resume_token_lower), university_combined)
    fourgram_university = get_university(get_fourgrams(resume_token_lower), university_combined)
    fivegram_university = get_university(get_fivegrams(resume_token_lower), university_combined)
    sixgram_university = get_university(get_sixgrams(resume_token_lower), university_combined)
    combined_university_extraction = set(bigram_university + threegram_university + fourgram_university + fivegram_university + sixgram_university)
    print (combined_university_extraction)

extract_university(resume_token2,university_combined)



{'university of minnesota', 'university of maryland'}


In [18]:
#extract GPA:
#use regular expression technique to extract gpa. works very well.

def extract_GPA(resume):
    result = re.search(r'(GPA|gpa): ?\d.\d{1,}',resume)
    if result:
        result = result.group(0)
    return (result)
    print (result)

extract_GPA(resume_file)

'GPA:3.43'

In [9]:
#extract majors:

'''
Same as extracting univerity, we use the extract matching technique to parse majors. 
We already have the n-grams for the resume file.
On later step, we can compare the n-grams resume list to the major list to get the matching result. 
'''


def extract_majors(a,b):
    majors=[]
    for x in a:
        if x in b:
            majors.append(x)
    return (majors)
    print (majors)
    
unigram_major = extract_majors(resume_token2, major_lower)  
bigram_major = extract_majors(get_bigrams(resume_token2), major_lower)
threegram_major = extract_majors(get_threegrams(resume_token2), major_lower)
combined_majors_list = unigram_major + bigram_major + threegram_major

# print out the extracted majors from the resume without redundancy. 
major_distinct = []
for i in combined_majors_list:
    if i not in major_distinct:
        major_distinct.append(i)
        
#print (combined_majors_list)
'''
As you can see in the result, it also contains information that are actually not major in my resume. 
Thus, we use the location (index) of those words in the resume file to further identify whether it's major or not. 
'''
print (major_distinct)



['education', 'business', 'information', 'management', 'accounting', 'finance', 'insurance', 'web', 'development', 'home', 'specialist', 'history', 'analyst', 'marketing', 'english', 'information system', 'accounting and finance']


In [10]:
#get major indexes 
#we get a dictionary of index for the previous major list that we derive from resume. 

dict = {'Name': 5}
for i, element in enumerate(major_distinct):
   x = resume_file2.find(element)
   dict[element] = x
   

del dict['Name']
print(dict)

{'information system': 220, 'accounting': 395, 'business': 144, 'english': 3176, 'marketing': 2670, 'development': 831, 'analyst': 2232, 'home': 986, 'history': 1158, 'management': 305, 'finance': 410, 'education': 108, 'information': 220, 'accounting and finance': 395, 'specialist': 1015, 'web': 735, 'insurance': 426}


In [11]:
#get indexes for specific terms

'''
We determine to find the location (index) of the following 4 specific terms: bachelor, master, minor, and university.
By comparing the indexes of resume's extracted major list to the 4 specific terms, we can get a much better filtered
list for majors.
'''

#bachelor
regular_expression = re.compile(r"/BA|BS|Bachelor of Science|Bachelor of Arts|BBA |B/A|Bachelor of Business Administration/", re.IGNORECASE)
bach_major_result = re.search(regular_expression, resume_file)
if bach_major_result:
   bach_major_result = bach_major_result.group()
print (bach_major_result)

if bach_major_result is not None:
    bach_major_index = resume_file.find(bach_major_result)
    print(bach_major_index)
    
#minor
regular_expression_two = re.compile(r"minor|Minor", re.IGNORECASE)
minor_result = re.search(regular_expression_two, resume_file)
if minor_result:
   minor_result = minor_result.group()
print (minor_result)

if minor_result is not None:
    minor_index = resume_file.find(minor_result)
    print(minor_index)

#master
regular_expression_three = re.compile(r"Master|master", re.IGNORECASE)
master_major_result = re.search(regular_expression_three, resume_file)
if master_major_result:
   master_major_result = master_major_result.group()
print (master_major_result)

if master_major_result is not None:
    master_major_index = resume_file.find(master_major_result)
    print(master_major_index)
    
#university
regular_expression_four = re.compile(r"university", re.IGNORECASE)
university_major_result = re.search(regular_expression_four, resume_file)
if university_major_result:
   university_major_result = university_major_result.group()
print (university_major_result)

if university_major_result is not None:
    university_major_index = resume_file.find(university_major_result)
    print(university_major_index)

Bachelor of Science
358
Minor
419
Master
199
University
154


In [12]:
'''
Continue with the previous topic, we try to find which majors words appear in the range (100 characters).
After the specific terms. If a major term is in that range, it would be saved into a list.
If it's not in the range, then the major term is discarded.
'''

upper_bound1 = bach_major_index+100
for k, v in dict.items():
   if (bach_major_index < v < upper_bound1):
       updated_majors1.append(k)
       indexes_majors1.append(v)
print(updated_majors1)
print(indexes_majors1)

upper_bound2 = master_major_index+100
for k, v in dict.items():
   if (master_major_index < v < upper_bound2):
       updated_majors2.append(k)
       indexes_majors2.append(v)
print(updated_majors2)
print(indexes_majors2)

upper_bound3 = university_major_index+100
for k, v in dict.items():
   if (university_major_index < v < upper_bound3):
       updated_majors3.append(k)
       indexes_majors3.append(v)
print(updated_majors3)
print(indexes_majors3)

upper_bound4 = minor_index+100
for k, v in dict.items():
   if (minor_index < v < upper_bound4):
       updated_majors4.append(k)
       indexes_majors4.append(v)
print(updated_majors4)
print(indexes_majors4)

['accounting', 'finance', 'accounting and finance', 'insurance']
[395, 410, 395, 426]
['information system', 'information']
[220, 220]
['information system', 'information']
[220, 220]
['insurance']
[426]


In [13]:
'''
With respect to the output of previous block, we use the fuzzywuzzy pacakge to get rid of redundancy. 
For example, if a list contains "accounting" and "accounting and finance", 
it would only keep the longer term which contains the shorter term.
'''
majors_minors_all = updated_majors1 + updated_majors2 + updated_majors3 + updated_majors4
majors_minors_final_list = list(dedupe(majors_minors_all))
print (majors_minors_final_list)

['information system', 'accounting and finance', 'insurance']
